# Imports

In [1]:
import pandas as pd
import numpy as np

---

In [2]:
events = pd.read_pickle("../../../../data/tp2/events_tp2_formateado.pkl")

In [3]:
events["dayofweek"] = (events["date"].dt.dayofweek).astype("int8")

---

# Armado de ventanas

In [4]:
TAMAÑO_VENTANA = 2
FECHA_INICIO = 17

In [5]:
def etiquetar_ventanas(dataset,col_fecha):
    for ventana_nro in range(1,6):
        
        fecha_inicio = FECHA_INICIO + ventana_nro
        fecha_fin = fecha_inicio + TAMAÑO_VENTANA
        
        dataset["ventana_{}".format(ventana_nro)] = (((fecha_inicio) <= dataset[col_fecha].dt.day) & ((dataset[col_fecha].dt.day) <= (fecha_fin)))
    return dataset    

In [6]:
events = etiquetar_ventanas(events,"date")

---

# Creación de features

## Creación de sets de entrenamiento

In [7]:
def crear_targets(dataframe,col_dispositivo):
    entrenamientos = {}
    for ventana_nro in range(1,6):
        ent = dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]][col_dispositivo].drop_duplicates().to_frame()
        ent.set_index(col_dispositivo, inplace=True)
        entrenamientos[ventana_nro] = ent
        
    return entrenamientos

In [8]:
Xs = crear_targets(events,"device_id")

---

## Función generadora de features

In [9]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos):
    
    for ventana_nro in range(1, 6):
        
        ventana = dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]]
        
        feature = generador_feature(ventana, ventana_nro)
        
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True)

In [15]:
#def calcular_features(ventanas,feature):
#    features = []
#    for x in range(5):
#        features.append(feature(ventanas[x]))
#    return features

In [16]:
#def mergear_features(targets,features):
#    for x in range(5):
#        targets[x] = targets[x].merge(features[x], how = 'left',left_index = True,right_index = True)


---

---

## Cantidad de eventos dentro de la ventna

In [10]:
def cantidad_eventos(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : "count"}).rename(columns={"date": "cant_eventos"})

In [11]:
generar_feature_en_ventanas(events,cantidad_eventos,Xs)

---

## Promedio de eventos por día

In [12]:
def prom_eventos(dataframe, nro_ventana):
    feature = dataframe.groupby(["device_id","dayofweek"]).agg({"date":"count"})
    feature = feature.groupby("device_id").agg({"date":"mean"})
    feature["date"] = feature["date"].astype("int64")
    return feature.rename(columns={"date": "prom_eventos"})

In [13]:
generar_feature_en_ventanas(events, prom_eventos, Xs)

---

## Cantidad de eventos discrimando periodos

In [14]:
# Creo columnas para luego separar
events['hora'] = events['date'].dt.hour
events['dia'] = events['date'].dt.day
events.rename(columns = {"dayofweek":"dia_de_semana"},inplace = True)

In [15]:
def cantidad_apariciones_por_hora(dataframe, nro_ventana):
    apariciones_por_hora = dataframe.groupby(["device_id", "hora"]).agg(({"date" : "count"}))
    apariciones_por_hora = apariciones_por_hora.unstack().fillna(0).astype(np.int64)
    apariciones_por_hora.columns = ["eventos_hora_{}".format(hora) for hora in range(0,24)]
    return apariciones_por_hora

In [16]:
generar_feature_en_ventanas(events, cantidad_apariciones_por_hora, Xs)

In [17]:
def cantidad_apariciones_por_dia(dataframe, nro_ventana):
    dia_final = TAMAÑO_VENTANA + 2
    
    apariciones_por_dia = dataframe.groupby(["device_id", "dia"]).agg(({"date" : "count"}))
    apariciones_por_dia = apariciones_por_dia.unstack().fillna(0).astype(np.int64)
    apariciones_por_dia.columns = ["eventos_dia_{}".format(dia) for dia in range(1,dia_final)]
    return apariciones_por_dia

In [18]:
generar_feature_en_ventanas(events, cantidad_apariciones_por_dia, Xs)

---

## Cantidad de veces que se realizó el evento más frecuente

In [21]:
def cantidad_maximo_evento(dataframe,nro_ventana):
    
    feature = events.groupby(["device_id","event_id"]).agg({"date":"count"})
    feature = feature.groupby("device_id").agg({"date":"max"})
    feature.rename(columns = {"date":"max_cant_eventos"},inplace = True)
    
    return feature
    

In [22]:
generar_feature_en_ventanas(events, cantidad_maximo_evento, Xs)

In [23]:
Xs[1].head()

,cant_eventos,prom_eventos,eventos_hora_0,eventos_hora_1,eventos_hora_2,eventos_hora_3,eventos_hora_4,eventos_hora_5,eventos_hora_6,eventos_hora_7,...,eventos_hora_18,eventos_hora_19,eventos_hora_20,eventos_hora_21,eventos_hora_22,eventos_hora_23,eventos_dia_1,eventos_dia_2,eventos_dia_3,max_cant_eventos
device_id,,,,,,,,,,,,,,,,,,,,,
4314242242878368557,10,3,7,0,0,0,0,0,0,0,...,0,0,1,0,0,0,6,3,1,5
2068670382964280534,140,70,138,0,0,0,0,0,0,0,...,1,1,0,0,0,0,138,0,2,36
726218951290512705,27,27,25,0,0,0,0,0,0,0,...,0,0,0,0,0,0,27,0,0,19
1329210878469536477,157,52,27,7,3,18,2,0,0,0,...,0,5,10,36,21,22,118,15,24,91
5661732594213028190,50,16,17,0,0,1,0,0,0,0,...,22,2,0,0,0,0,17,1,32,23


---

# Guardado de los sets de entrenamiento

In [24]:
for ventana_nro in range(1, 6):
    Xs[ventana_nro].to_pickle("../../features/events_train_ventana_{}.pkl".format(ventana_nro))
    Xs[ventana_nro].to_csv("../../features/events_train_ventana_{}.csv".format(ventana_nro))

# ACA SE DEBE CONTINUAR

-----

---

---

## Cuantos eventos comparten la primera posición de evento más frecuente

In [22]:
def cant_eventos_maximos(ventana):
    feature = ventana.groupby(["ref_hash","event_id"]).size().to_frame()
    idx = feature.groupby(['ref_hash'])[0].transform(max) == feature[0]
    feature = feature[idx]
    feature = feature.groupby("ref_hash")[0].count().to_frame()
    feature.rename(columns= {0:"cant_eventos_maximos"},inplace = True)
    return feature    
    
    

In [23]:
features = calcular_features(ventanas,cant_eventos_maximos)

In [24]:
mergear_features(targets,features)

In [25]:
e1 = targets[0]

In [26]:
e1.head()

,promedio_eventos,max_cant_event,cant_eventos_maximos
device_id,,,
4314242242878368557,3,5,1
2068670382964280534,70,36,1
726218951290512705,27,16,1
1329210878469536477,52,65,1
5661732594213028190,16,23,1


In [27]:
e1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166002 entries, 4314242242878368557 to 8658033545182476382
Data columns (total 3 columns):
promedio_eventos        166002 non-null int64
max_cant_event          166002 non-null int64
cant_eventos_maximos    166002 non-null int64
dtypes: int64(3)
memory usage: 10.1 MB


---

---

## Último evento

#### Pruebo primero con una ventana

In [ ]:
ventana1 = ventanas[0]

In [244]:
idx = ventana1.groupby(['ref_hash'])["date"].transform(max) == ventana1["date"]
feature = ventana1[idx]

In [245]:
len(feature)

166018

In [247]:
feature["ref_hash"].nunique()

166002

### Hay valores que tienen la misma fecha con el mismo evento, pero distinto kind, por ejemplo: 

In [248]:
feature[feature["ref_hash"] == 2831641632794815413]

,date,event_id,ref_hash,application_id,attributed,session_user_agent,kind,wifi,os,dayofweek
7488551,2019-04-20 06:55:29.193,109,2831641632794815413,116,False,5.783151e+18,1.020653e+18,False,0,5
7488552,2019-04-20 06:55:29.193,108,2831641632794815413,116,False,5.783151e+18,2.376750e+18,False,0,5


#### Solo me importa el evento así que descarto al azar

In [249]:
feature = feature[["ref_hash","date","event_id","application_id"]].drop_duplicates("ref_hash")
feature.rename(columns = {"ref_hash":"device_id","event_id":"ult_ev","date":"fecha_ult_ev","application_id":"ult_app"},inplace = True)
feature.set_index("device_id",inplace = True)

In [250]:
feature.head()

,fecha_ult_ev,ult_ev,ult_app
device_id,,,
6444651396290009624,2019-04-18 00:00:02.052,245,21
3601178220858933758,2019-04-18 00:00:07.957,246,21
3233293871630648142,2019-04-18 00:00:09.612,15,65
4452972490570146531,2019-04-18 00:00:12.303,7,122
5451417563610992673,2019-04-18 00:00:19.597,106,121


In [253]:
def ult_evento(ventana):

    idx = ventana.groupby(['ref_hash'])["date"].transform(max) == ventana["date"]
    feature = ventana[idx]
    feature = feature[["ref_hash","date","event_id","application_id"]].drop_duplicates("ref_hash")
    feature.rename(columns = {"ref_hash":"device_id","event_id":"ult_ev","date":"fecha_ult_ev","application_id":"ult_app"},inplace = True)
    feature.set_index("device_id",inplace = True)
    return feature

In [254]:
features = calcular_features(ventanas,ult_evento)

In [255]:
mergear_features(targets,features)

In [256]:
e1 = targets[0]

In [257]:
e1.head()

,promedio_eventos,max_cant_event,cant_eventos_maximos,fecha_ult_ev,ult_ev,ult_app
device_id,,,,,,
4314242242878368557,3,5,1,2019-04-20 20:03:34.247,15,65
2068670382964280534,70,36,1,2019-04-20 19:41:28.313,15,49
726218951290512705,27,16,1,2019-04-18 13:39:15.767,15,65
1329210878469536477,52,65,1,2019-04-20 23:39:14.726,508,2
5661732594213028190,16,23,1,2019-04-20 19:36:04.032,2,210


# HASTA ACÁ ES LO ÚLTIMO QUE LLEGUE, LO QUE SIGUE SON COSAS VIEJAS QUE TODAVIA NO LAS ADAPTE 

## Aplicación más usada

In [19]:
feature = ventana1.groupby(['ref_hash',"application_id"]).size().to_frame()

In [25]:
feature.head()

0
ref_hash        application_id    
41863526108385  65              21
                121              3
                210             64
161514654074162 121              8
186034136943920 210             13

In [29]:
idx = feature.groupby(['ref_hash'])[0].transform(max) == feature[0]
feature = feature[idx].reset_index()

In [30]:
feature["ref_hash"].nunique()

166002

In [31]:
len(feature)

168104

### Propuesta: Hay valores repetidos, podría quedarme con la hora más cercana a las 00 hs y calcular el tiempo que falta desde t0 a esa hora. Además de indicar la cantidad de eventos registrados a esa hora, y la cantidad de horas que comparten el primer lugar.

### Propuesta: En vez de indicar valores categoricos como aplicación más usada, evento más registrado, se podría indicar cual es su valor de frecuencia, ejemplo:
#### __Freq_ev_popular__ : 53
#### __Freq_app_popular__ : 12

### Propuesta : En los valores en donde se comparten horas, eventos, aplicaciones, etc, como los más vistos, se podría indicar cual es la cantidad de valores que comparten ese puesto, ejemplo:

#### __cant_horas_mas_activas__: 2
#### __cant_events_en_horas_mas_activas__ : 10